<a href="https://colab.research.google.com/github/PondKann/KKU_data_mining/blob/master/Data_Retail2_Association.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Association All Country

In [1]:
import pandas as pd
import os 
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from google.colab import drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
path = '/content/drive/My Drive/DataViz_2021'
os.listdir(path)

['pm-18-jan-2021.csv',
 'covid-19-daily_data_dictionary.xlsx',
 'income.csv',
 'income2.xls',
 'จัดซื้อ 57-58.xls',
 'online_retail_II.xlsx',
 'online_retail_II.csv']

1. นำเข้าข้อมูล และ Package ที่ต้องการ

In [4]:
data_path = os.path.join(path,'online_retail_II.csv')
datacsv = pd.read_csv(os.path.join(path,'online_retail_II.csv'),encoding='utf-8')
df = datacsv

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541910 entries, 0 to 541909
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Invoice      541910 non-null  object 
 1   StockCode    541910 non-null  object 
 2   Description  540456 non-null  object 
 3   Quantity     541910 non-null  int64  
 4   InvoiceDate  541910 non-null  object 
 5   Price        541910 non-null  float64
 6   Customer ID  406830 non-null  float64
 7   Country      541910 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [8]:
df

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,12/9/2011 12:50,4.95,12680.0,France


2. ทำความสะอาดข้อมูล โดยการลบรายการที่ถูกยกเลิกออก

In [9]:
df['Description'] = df['Description'].str.strip()
df.dropna(axis=0, subset=['Invoice'], inplace=True)
df['Invoice'] = df['Invoice'].astype('str')
df = df[~df['Invoice'].str.contains('C')]

3. Group by ข้อมูล Invoice และ Description โดยให้แสดงจำนวนสินค้าที่ถูกสั่งในแต่ละรายการ (Invoice) ถ้าไม่ได้สั่งสินค้านั้นก็ให้ใส่เลขศูนย์

In [10]:
basket0 = (df.groupby(['Invoice', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('Invoice'))

4. แปลงค่าข้อมูลในตาราง basket0 ถ้าค่าข้อมูลเป็น <= 0 ให้เปลี่ยนเป็นเลข 0 แต่ถ้าค่าข้อมูลเป็น >= 1 ให้เปลี่ยนเป็นเลข 1 และลบ รหัสไปรษณีย์ (POSTAGE)

In [7]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1


5. กฎความสัมพันธ์ โดยกำหนด confidence >= 0.8

In [12]:
basket0_sets = basket0.applymap(encode_units)
basket0_sets.drop('POSTAGE', inplace=True, axis=1)
frequent_itemsets0 = apriori(basket0_sets, min_support=0.018, use_colnames=True)
rules0 = association_rules(frequent_itemsets0, metric="lift", min_threshold=1)

AssoAll = rules0[ (rules0['lift'] >= 4)& (rules0['confidence'] >= 0.8)]

In [13]:
AssoAll

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
35,(PINK REGENCY TEACUP AND SAUCER),(GREEN REGENCY TEACUP AND SAUCER),0.037166,0.049248,0.030713,0.826371,16.779804,0.028883,5.475760
209,"(GREEN REGENCY TEACUP AND SAUCER, PINK REGENCY...",(ROSES REGENCY TEACUP AND SAUCER),0.030713,0.051722,0.026298,0.856240,16.554511,0.024709,6.596260
210,"(ROSES REGENCY TEACUP AND SAUCER, PINK REGENCY...",(GREEN REGENCY TEACUP AND SAUCER),0.029064,0.049248,0.026298,0.904841,18.373184,0.024867,9.991237
214,"(GREEN REGENCY TEACUP AND SAUCER, REGENCY CAKE...",(ROSES REGENCY TEACUP AND SAUCER),0.024697,0.051722,0.019796,0.801572,15.497554,0.018519,4.778943
220,"(JUMBO BAG PINK POLKADOT, JUMBO SHOPPER VINTAG...",(JUMBO BAG RED RETROSPOT),0.022707,0.101504,0.018341,0.807692,7.957236,0.016036,4.672179
226,"(JUMBO BAG PINK POLKADOT, JUMBO STORAGE BAG SUKI)",(JUMBO BAG RED RETROSPOT),0.024988,0.101504,0.020039,0.801942,7.900583,0.017502,4.536523


# Association Some Country

In [26]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [32]:
dft = df.iloc[:5,:]
dft

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom


In [33]:
df['Sales'] = df['Quantity'] * df['Price']
df_new = df
df_new

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Sales
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom,20.34
...,...,...,...,...,...,...,...,...,...
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12/9/2011 12:50,2.10,12680.0,France,12.60
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12/9/2011 12:50,4.15,12680.0,France,16.60
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12/9/2011 12:50,4.15,12680.0,France,16.60
541908,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,12/9/2011 12:50,4.95,12680.0,France,14.85


In [34]:
df_country = df_new.groupby('Country').sum().sort_values('Sales', ascending=False)
df_country

,Quantity,Price,Customer ID,Sales
Country,,,,
United Kingdom,4263829,2.245715e+06,5.626433e+09,8.187806e+06
Netherlands,200128,6.492550e+03,3.419054e+07,2.846615e+05
EIRE,142637,4.844719e+04,1.103917e+08,2.632768e+05
Germany,117448,3.766600e+04,1.200751e+08,2.216982e+05
France,110481,4.304999e+04,1.076615e+08,1.974219e+05
Australia,83653,4.054750e+03,1.569300e+07,1.370773e+05
Switzerland,30325,6.813690e+03,2.377592e+07,5.638535e+04
Spain,26824,1.263345e+04,3.268929e+07,5.477458e+04
Belgium,23152,7.540130e+03,2.571829e+07,4.091096e+04


In [35]:
df_country.to_csv('df_country.csv')

## United Kingdom

In [30]:
basket1 = (df[df['Country'] =="United Kingdom"]
          .groupby(['Invoice', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('Invoice'))

basket_sets1 = basket1.applymap(encode_units)
basket_sets1.drop('POSTAGE', inplace=True, axis=1)
frequent_itemsets1 = apriori(basket_sets1, min_support=0.025, use_colnames=True)
rules1 = association_rules(frequent_itemsets1, metric="lift", min_threshold=1)

asso_UK =  rules1[ (rules1['lift'] >= 4) &
        (rules1['confidence'] >= 0.7)]

In [31]:
asso_UK

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
2,(PINK REGENCY TEACUP AND SAUCER),(GREEN REGENCY TEACUP AND SAUCER),0.031897,0.042377,0.026180,0.820768,19.368019,0.024828,5.342926
4,(ROSES REGENCY TEACUP AND SAUCER ),(GREEN REGENCY TEACUP AND SAUCER),0.043421,0.042377,0.031806,0.732497,17.285056,0.029966,3.579862
5,(GREEN REGENCY TEACUP AND SAUCER),(ROSES REGENCY TEACUP AND SAUCER ),0.042377,0.043421,0.031806,0.750535,17.285056,0.029966,3.834527


In [39]:
asso_UK.to_csv('asso_UK.csv')

## Netherlands

In [24]:
basket2 = (df[df['Country'] =="Netherlands"]
          .groupby(['Invoice', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('Invoice'))

basket_sets2 = basket2.applymap(encode_units)
basket_sets2.drop('POSTAGE', inplace=True, axis=1)
frequent_itemsets2 = apriori(basket_sets2, min_support=0.08, use_colnames=True)
rules2 = association_rules(frequent_itemsets2, metric="lift", min_threshold=1)

asso_NTL = rules2[ (rules2['lift'] >= 5) &
        (rules2['confidence'] >= 0.9)]

In [25]:
asso_NTL

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
209,"(DOLLY GIRL LUNCH BOX, RED RETROSPOT CHARLOTTE...","(SPACEBOY LUNCH BOX , ROUND SNACK BOXES SET OF...",0.089109,0.178218,0.089109,1.000000,5.611111,0.073228,inf
211,"(SPACEBOY LUNCH BOX , RED RETROSPOT CHARLOTTE ...","(DOLLY GIRL LUNCH BOX, ROUND SNACK BOXES SET O...",0.089109,0.158416,0.089109,1.000000,6.312500,0.074993,inf
222,"(DOLLY GIRL LUNCH BOX, ROUND SNACK BOXES SET O...","(SPACEBOY LUNCH BOX , ROUND SNACK BOXES SET OF...",0.108911,0.178218,0.099010,0.909091,5.101010,0.079600,9.039604


In [26]:
asso_NTL.to_csv('asso_NTL.csv')

## Germany

In [16]:
basket2 = (df[df['Country'] =="Germany"]
          .groupby(['Invoice', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('Invoice'))

basket_sets2 = basket2.applymap(encode_units)
basket_sets2.drop('POSTAGE', inplace=True, axis=1)
frequent_itemsets2 = apriori(basket_sets2, min_support=0.05, use_colnames=True)
rules2 = association_rules(frequent_itemsets2, metric="lift", min_threshold=1)

rules2[ (rules2['lift'] >= 4) &
        (rules2['confidence'] >= 0.5)]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1,(PLASTERS IN TIN CIRCUS PARADE),(PLASTERS IN TIN WOODLAND ANIMALS),0.115974,0.137856,0.067834,0.584906,4.242887,0.051846,2.076984
7,(PLASTERS IN TIN SPACEBOY),(PLASTERS IN TIN WOODLAND ANIMALS),0.107221,0.137856,0.061269,0.571429,4.145125,0.046488,2.011670
11,(RED RETROSPOT CHARLOTTE BAG),(WOODLAND CHARLOTTE BAG),0.070022,0.126915,0.059081,0.843750,6.648168,0.050194,5.587746
